In [1]:
import sys
import os
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, AgglomerativeClustering, dbscan

from mlxtend.frequent_patterns import apriori, association_rules

import hdbscan

%matplotlib inline

In [2]:
os.chdir("L:/lab_research/RES-Folder-UPOD/Celldynclustering")
celldyn = pd.read_sas("E_ResearchData/2_ResearchData/celldyn.sas7bdat")

D:\Miniconda3\envs\embcelldyn\lib\site-packages\pandas\io\sas\sas7bdat.py:800: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  rslt[name] = self._byte_chunk[jb, :].view(dtype=self.byte_order + "d")
D:\Miniconda3\envs\embcelldyn\lib\site-packages\pandas\io\sas\sas7bdat.py:809: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  rslt[name] = self._string_chunk[js, :]


In [ ]:
celldyn.columns

In [3]:
celldyn['afnamejaar']  = pd.to_datetime(celldyn.afname_dt).dt.year

<ipython-input-3-0dac9241b46a>:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  celldyn['afnamejaar']  = pd.to_datetime(celldyn.afname_dt).dt.year


In [4]:
meas_columns = [c for c in celldyn.columns if 'c_b' in c]
mode_columns = [c for c in celldyn.columns if 'c_m' in c]
alrt_columns = [c for c in celldyn.columns if 'alrt' in c.lower()]
c_s_columns = [c for c in celldyn if 'c_s_' in c.lower()]

In [6]:
for c in c_s_columns:
    print(c,len(celldyn.loc[celldyn[c] == 0]))

c_s_wbc 152
c_s_wvf 152
c_s_neu 152
c_s_seg 152
c_s_bnd 152
c_s_ig 152
c_s_lym 152
c_s_lyme 152
c_s_vlym 152
c_s_mon 152
c_s_mone 152
c_s_blst 152
c_s_eos 152
c_s_bas 152
c_s_nrbc 152
c_s_pneu 152
c_s_pseg 152
c_s_pbnd 152
c_s_pig 152
c_s_plym 152
c_s_plyme 152
c_s_pvlym 152
c_s_pmon 152
c_s_pmone 152
c_s_pblst 152
c_s_peos 152
c_s_pbas 152
c_s_pnrbc 152
c_s_rbci 152
c_s_rbco 152
c_s_hb 152
c_s_mcv 152
c_s_rdw 152
c_s_mch 152
c_s_mchc 152
c_s_ht 152
c_s_plt 155
c_s_plto 152
c_s_plti 152
c_s_mpv 152
c_s_pct 152
c_s_pdw 152
c_s_retc 613135
c_s_pretc 613135
c_s_irf 613135


In [5]:
cluster_columns = c_s_columns+alrt_columns
celldyn = celldyn[cluster_columns].dropna()
clust_data = celldyn.sample(250000)

### try ARM

In [6]:
fin_c_s_cols = ['c_s_wbc', 'c_s_wvf', 'c_s_neu', 'c_s_seg', 'c_s_lym','c_s_vlym', 'c_s_mon', 'c_s_mone', 'c_s_blst',
       'c_s_eos', 'c_s_bas', 'c_s_nrbc', 'c_s_rbci', 'c_s_rbco', 'c_s_hb', 'c_s_mcv', 'c_s_rdw', 'c_s_mch',
       'c_s_mchc', 'c_s_ht', 'c_s_plt', 'c_s_plto', 'c_s_plti', 'c_s_mpv',
       'c_s_pct', 'c_s_pdw', 'c_s_retc', 'c_s_pretc', 'c_s_irf']

In [7]:
with open("E_ResearchData/2_ResearchData/arm_index_to_take.txt",'r') as f:
    index_to_take = [int(x) for x in f.read().split("\n")[:-1]]

### c_s_ != 1


Bekijk associaties tussen alert en suspect vlaggen als ze niet OK zijn (c_s_ != 1, alrt == 1)

In [53]:
arm_data = celldyn.copy()
arm_data[alrt_columns] = np.where(arm_data[alrt_columns] == 1,True,False)
arm_data[c_s_columns] = np.where(arm_data[c_s_columns] != 1,True,False)
frequent_candidates = apriori(arm_data[c_s_columns+alrt_columns], min_support=0.01, use_colnames=True,low_memory=True)

rules = association_rules(frequent_candidates, metric="lift", min_threshold=1)
interesting_rules = rules[ (rules['confidence'] >= 0.8) ]
c_s_isno1 = ((tuple(k),tuple(v),np.round(x,decimals=3),np.round(y,decimals = 3)) for k,v,x,y in zip(interesting_rules.antecedents,
                                                           interesting_rules.consequents,
                                                           interesting_rules.support,
                                                           interesting_rules.confidence))
c_s_isno1

MemoryError: Unable to allocate 20.7 MiB for an array with shape (7, 3099542) and data type bool

In [54]:
[x for x in c_s_isno1]

[]

### c_s_ = 2

Check als c_s_ kolommen 2 geven, dit ook gelijk 2 is bij andere c_s_ kolommen

In [ ]:
arm_data = celldyn.copy()
arm_data[alrt_columns] = np.where(arm_data[alrt_columns] == 1,True,False)
arm_data[c_s_columns] = np.where(arm_data[c_s_columns] == 2,True,False)
frequent_candidates = apriori(arm_data[c_s_columns+alrt_columns], min_support=0.01, use_colnames=True,low_memory=True)

rules = association_rules(frequent_candidates, metric="lift", min_threshold=1)
interesting_rules = rules[ (rules['confidence'] >= 0.8) ]
c_s_is2 = ((tuple(k),tuple(v),np.round(x,decimals=3),np.round(y,decimals = 3)) for k,v,x,y in zip(interesting_rules.antecedents,
                                                           interesting_rules.consequents,
                                                           interesting_rules.support,
                                                           interesting_rules.confidence))
[x for x in c_s_is2]

### c_s = 4

Check als c_s_ kolommen 4 geven, dit ook gelijk 4 is bij andere c_s_ kolommen

In [25]:
arm_data = celldyn.copy()
arm_data[alrt_columns] = np.where(arm_data[alrt_columns] == 1,True,False)
arm_data[c_s_columns] = np.where(arm_data[c_s_columns] == 4,True,False)
frequent_candidates = apriori(arm_data[c_s_columns+alrt_columns], min_support=0.05, use_colnames=True,low_memory=True,)

rules = association_rules(frequent_candidates, metric="lift", min_threshold=1)
interesting_rules = rules[ (rules['confidence'] >= 0.8) ]
c_s_is4 = ((tuple(k),tuple(v),np.round(x,decimals=3),np.round(y,decimals = 3)) for k,v,x,y in zip(interesting_rules.antecedents,
                                                           interesting_rules.consequents,
                                                           interesting_rules.support,
                                                           interesting_rules.confidence))
[x for x in c_s_is4]

[]

### c_s = 5

Check als c_s_ kolommen 5 geven, dit ook gelijk 5 is bij andere c_s_ kolommen

In [42]:
[arm_data[x].value_counts()/len(arm_data) for x in arm_data]

[False    1.0
 Name: c_s_wbc, dtype: float64,
 False    1.0
 Name: c_s_wvf, dtype: float64,
 False    0.885924
 True     0.114076
 Name: c_s_neu, dtype: float64,
 False    1.0
 Name: c_s_seg, dtype: float64,
 False    1.0
 Name: c_s_bnd, dtype: float64,
 False    1.0
 Name: c_s_ig, dtype: float64,
 False    0.954162
 True     0.045838
 Name: c_s_lym, dtype: float64,
 False    1.0
 Name: c_s_lyme, dtype: float64,
 False    1.0
 Name: c_s_vlym, dtype: float64,
 False    0.948107
 True     0.051893
 Name: c_s_mon, dtype: float64,
 False    1.0
 Name: c_s_mone, dtype: float64,
 False    1.0
 Name: c_s_blst, dtype: float64,
 False    1.0
 Name: c_s_eos, dtype: float64,
 False    1.0
 Name: c_s_bas, dtype: float64,
 False    0.968248
 True     0.031752
 Name: c_s_nrbc, dtype: float64,
 False    0.885924
 True     0.114076
 Name: c_s_pneu, dtype: float64,
 False    1.0
 Name: c_s_pseg, dtype: float64,
 False    1.0
 Name: c_s_pbnd, dtype: float64,
 False    1.0
 Name: c_s_pig, dtype: float64,

In [51]:
arm_data = celldyn.copy()
arm_data[alrt_columns] = np.where(arm_data[alrt_columns] == 1,True,False)
arm_data[c_s_columns] = np.where(arm_data[c_s_columns] == 5,True,False)
frequent_candidates = apriori(arm_data[c_s_columns+alrt_columns], min_support=0.01, use_colnames=True,low_memory=True)

rules = association_rules(frequent_candidates, metric="lift", min_threshold=1)
interesting_rules = rules[ (rules['confidence'] >= 0.8) ]
c_s_is5 = ((tuple(k),tuple(v),np.round(x,decimals=3),np.round(y,decimals = 3)) for k,v,x,y in zip(interesting_rules.antecedents,
                                                           interesting_rules.consequents,
                                                           interesting_rules.support,
                                                           interesting_rules.confidence))
c_s_is5 = [x for x in c_s_is5]

In [52]:
with open("L:\laupodteam\AIOS\Huibert-Jan\Celldynclustering\c_s_is5.txt",'w') as f:
    for line in c_s_is5:
        f.write(f'{line[0]};{line[1]};{line[2]};{line[-1]}'+'\n')

### try permutations of c_s_ = {2,4,5}.
Set 1 on 2,4 or 5, and see if other columns not on that number combine
SO see if you put 1 column on 2, other columns are always 4 or 5, and do this the other way around as well

In [230]:
for col in c_s_columns:
    arm_data = celldyn.loc[index_to_take]
    arm_data[alrt_columns] = np.where(arm_data[alrt_columns] == 1,True,False)
    arm_data[col] = np.where(arm_data[col] == 5,True,False)
    other_cols = [c for c in c_s_columns if c != col]
    arm_data[other_cols] = np.where(arm_data[other_cols] == 4,True,False)
    frequent_candidates = apriori(arm_data[c_s_columns+alrt_columns], min_support=0.05, use_colnames=True,low_memory=True)
    rules = association_rules(frequent_candidates, metric="lift", min_threshold=1)
    interesting_rules = rules[ (rules['confidence'] >= 0.8) ]
    print(col,{tuple(k):tuple(v) for k,v in zip(interesting_rules.antecedents,interesting_rules.consequents) if tuple(k) not in c_s_is4.keys() and tuple(k) not in c_s_is5.keys()})

c_s_wbc {}
c_s_wvf {}
c_s_neu {}
c_s_seg {}
c_s_bnd {}
c_s_ig {}
c_s_lym {}
c_s_lyme {}
c_s_vlym {}
c_s_mon {}
c_s_mone {}
c_s_blst {}
c_s_eos {}
c_s_bas {}
c_s_nrbc {}
c_s_pneu {}
c_s_pseg {}
c_s_pbnd {}
c_s_pig {}
c_s_plym {}
c_s_plyme {}
c_s_pvlym {}
c_s_pmon {}
c_s_pmone {}
c_s_pblst {}
c_s_peos {}
c_s_pbas {}
c_s_pnrbc {}
c_s_rbci {}
c_s_rbco {}
c_s_hb {}
c_s_mcv {}
c_s_rdw {}
c_s_mch {}
c_s_mchc {}
c_s_ht {}
c_s_plt {}
c_s_plto {}
c_s_plti {}
c_s_mpv {}
c_s_pct {}
c_s_pdw {}
c_s_retc {}
c_s_pretc {}
c_s_irf {}


### try PCA

In [ ]:
var_ex = []
for i in range(2,20):
    pca = PCA(n_components=i)
    pca.fit(celldyn[cluster_columns].dropna())
    var_ex.append(pca.explained_variance_)
    
    


In [ ]:
pca = PCA(n_components = 8)
transformed_data = pca.fit_transform(clust_data.dropna())

In [ ]:
km = KMeans(n_clusters=3).fit(transformed_data)

In [ ]:
celldyn = celldyn.dropna()
celldyn['km_labels']  = km.labels_

### check clusters after pca

In [ ]:
plot_df = pd.DataFrame(data=transformed_data, columns=[f'd_{i}' for i in range(8)], index=clust_data.index)
plot_df['labels'] = hdb_cl.labels_
#reduce_dim = min([6, reduce_dim])


In [ ]:
cols_to_plot = ['c_s_wbc', 'c_s_wvf', 'c_s_neu', 'c_s_seg', 'c_s_lym','c_s_vlym', 'c_s_mon', 'c_s_mone', 'c_s_blst',
       'c_s_eos', 'c_s_bas', 'c_s_nrbc','c_s_rbci', 'c_s_rbco', 'c_s_hb', 'c_s_mcv', 'c_s_rdw', 'c_s_mch','c_s_mchc', 'c_s_ht',
        'c_s_plt', 'c_s_plto', 'c_s_plti', 'c_s_mpv','c_s_pct', 'c_s_pdw', 'c_s_retc', 'c_s_pretc', 'c_s_irf']

In [ ]:
pd.set_option("display.max_columns",50)
clust_data[c_s_columns].corr()

In [ ]:
len(cols_to_plot)

In [ ]:
num_rows = int(np.ceil(len(cols_to_plot)*len(cols_to_plot)/3))
fig, ax = plt.subplots(ncols=3, nrows=num_rows, figsize=(18, 3*num_rows))
k = 0
rest_cols = cols_to_plot[1:]

for ki in range(len(cols_to_plot)):
    rest_cols = rest_cols[1:]
    for kj in range(len(rest_cols)): 
        i = int(k/3)
        j = k%3
        if num_rows > 1:
            sns.scatterplot(data=clust_data.sample(20000), x=cols_to_plot[ki], y=rest_cols[kj],
                            ax=ax[i,j], 
                            hue = 'labels',alpha=0.5)
        else:
            sns.scatterplot(data=clust_data.sample(20000),x=cols_to_plot[ki], y=rest_cols[kj],
                            ax=ax[k], 
                             hue = 'labels',alpha=0.5)
        k += 1
        